In [5]:
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print('running target()')
    
target()
print(target)

running inner()
<function deco.<locals>.inner at 0x01986978>


# 使用装饰器改进“策略”模式

> 函数装饰器在导入模块时立即执行， 而被装饰的
函数只在明确调用时运行

In [6]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x01986588>)
running register(<function f2 at 0x019866A8>)
running main()
registry -> [<function f1 at 0x01986588>, <function f2 at 0x019866A8>]
running f1()
running f2()
running f3()


> 用装饰器改进策略模式

In [7]:
from collections import namedtuple

promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() *.1

    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)


# 顾客名称和积分
Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())




joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)]


banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]



print("开始使用best打折策略：")
print(Order(joe, long_order, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(ann, cart, best_promo))

开始使用best打折策略：
<Order total: 10.00 due: 9.30
<Order total: 30.00 due: 28.50
<Order total: 42.00 due: 39.90


# 变量作用域规则

In [18]:
b = 6
print(id(b))
def f2(a):
    print(a)
    print(id(b))
    print(b)

def f3(a):
    print(a)
    b = 9
    print(id(b))
    print(b)
    
    
def f4(a):
    print(a)
    # 没有绑定
    print(b)
    b = 9   
# Python 编译函数的定义体时， 它判断 b 是局部变量， 因为在函数中给它赋值了。 
# 生成的字节码证实了这种判断， Python 会尝试从本地环境获取 b。 后面调用 f2(3) 时， 
# f2 的定义体会获取并打印局部变量 a 的值， 但是尝试获取局部变量 b 的值时， 发现 b 没有绑定值。     

### 在函数内部出现了 b =9 这样的语句，b就是一个局部变量，在函数内部使用b之前必须先对b赋值，f3与f4的区别
### 注意id(b)的输出，发f2与f3的区别

f2(3)
print("***************************************")
f3(3)
print("***************************************")
f4(3)

1659660496
3
1659660496
6
***************************************
3
1659660544
9
***************************************
3


UnboundLocalError: local variable 'b' referenced before assignment

# global

In [20]:
b = 6
print(id(b))
def f3(a):
    global b
    print(id(b))
    print(a)
    print(b)
    b = 9
    print(id(b))
f3(3)    
print(id(b))  
print(b)

1659660496
1659660496
3
6
1659660544
1659660544
9


# 闭包

> 闭包是一种函数， 它会保留定义函数时存在的自由变量的绑定，
这样调用函数时， 虽然定义作用域不可用了， 但是仍能使用那些绑定

In [24]:
class Average():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Average()
print(avg(10),avg(11),avg(12))

10.0 10.5 11.0


In [30]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = make_averager()
print(avg(10), avg(11), avg(12))
for i in range(10):
    print(avg(i))

10.0 10.5 11.0
8.25
6.8
6.0
5.571428571428571
5.375
5.333333333333333
5.4
5.545454545454546
5.75
6.0


In [31]:
print(avg.__code__.co_varnames)
print('自由变量',avg.__code__.co_freevars)
print(avg.__closure__,avg.__closure__[0].cell_contents)
# __closure__始终保留着seri的真正值

('new_value', 'total')
自由变量 ('series',)
(<cell at 0x01A24910: list object at 0x01CE4760>,) [10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


# nolocal声明

In [36]:
def make_average():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count
        nonlocal total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_average()
avg (10)
##当 count 是数字或任何不可变类型时， count += 1 语句的作用其实与 count = count + 1 一样。 因此， 我们在 averager 的定义
#体中为 count 赋值了， 这会把 count 变成局部变量.对数字、 字符串、 元组等不可变类型来说， 只能读取， 不能更新。
#如果尝试重新绑定， 例如 count = count + 1， 其实会隐式创建局部
#变量 count。 这样， count 就不是自由变量了， 因此不会保存在闭包中


10.0

# 实现一个简单的装饰器

In [39]:
import time 
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args) ## 闭包
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8f秒] %s (%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n <2 else n * factorial(n-1)

if __name__ == '__main__':
    print("*" * 40, 'Calling snooze(.123)')
    snooze(.123)
    print("*" * 40, 'Calling factorial(6)')
    factorial(6)

**************************************** Calling snooze(.123)
[0.12275116秒] snooze (0.123) -> None 
**************************************** Calling factorial(6)
[0.00000079秒] factorial (1) -> 1 
[0.00004622秒] factorial (2) -> 2 
[0.00008217秒] factorial (3) -> 6 
[0.00009916秒] factorial (4) -> 24 
[0.00011654秒] factorial (5) -> 120 
[0.00013314秒] factorial (6) -> 720 


# 标准库里的装饰器

> functools.lru_cache
>> lru --> Least Recently Used  这是一项优化技术， 它把耗时的函数的结果保存起来， 避免传入相同的参数时重复计算.缓存不会无限制增长， 一段时间不用的缓存条目会被扔掉。

In [40]:
@clock
def fibonacci(n):
    if n < 2:
        return n 
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))
# 显然f(0) f(1)等都被多次调用执行，效率低

[0.00000079秒] fibonacci (0) -> 0 
[0.00000079秒] fibonacci (1) -> 1 
[0.00033699秒] fibonacci (2) -> 1 
[0.00000040秒] fibonacci (1) -> 1 
[0.00000040秒] fibonacci (0) -> 0 
[0.00000040秒] fibonacci (1) -> 1 
[0.00003200秒] fibonacci (2) -> 1 
[0.00009916秒] fibonacci (3) -> 2 
[0.00046933秒] fibonacci (4) -> 3 
[0.00000040秒] fibonacci (1) -> 1 
[0.00001738秒] fibonacci (0) -> 0 
[0.00000040秒] fibonacci (1) -> 1 
[0.00004741秒] fibonacci (2) -> 1 
[0.00007704秒] fibonacci (3) -> 2 
[0.00000040秒] fibonacci (0) -> 0 
[0.00000040秒] fibonacci (1) -> 1 
[0.00003872秒] fibonacci (2) -> 1 
[0.00000040秒] fibonacci (1) -> 1 
[0.00000237秒] fibonacci (0) -> 0 
[0.00000040秒] fibonacci (1) -> 1 
[0.00004662秒] fibonacci (2) -> 1 
[0.00008652秒] fibonacci (3) -> 2 
[0.00016474秒] fibonacci (4) -> 3 
[0.00027575秒] fibonacci (5) -> 5 
[0.00077946秒] fibonacci (6) -> 8 
8


## lru --> Least Recently Used 

In [42]:
import functools

# 叠放了装饰器： @lru_cache() 应用到 @clock 返回的函数上
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n 
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000079秒] fibonacci (0) -> 0 
[0.00000079秒] fibonacci (1) -> 1 
[0.00027457秒] fibonacci (2) -> 1 
[0.00000158秒] fibonacci (3) -> 2 
[0.00032909秒] fibonacci (4) -> 3 
[0.00000119秒] fibonacci (5) -> 5 
[0.00038163秒] fibonacci (6) -> 8 
8


```
functools.lru_cache(maxsize=128, typed=False)
maxsize 参数指定存储多少个调用的结果。 缓存满了之后， 旧的结果会
被扔掉， 腾出空间。 为了得到最佳性能， maxsize 应该设为 2 的
幂。 typed 参数如果设为 True， 把不同参数类型得到的结果分开保存， 
即把通常认为相等的浮点数和整数参数（如 1 和 1.0） 区分开。 顺
便说一下， 因为 lru_cache 使用字典存储结果， 而且键根据调用时传
入的定位参数和关键字参数创建， 所以被 lru_cache 装饰的函数， 它
的所有参数都必须是可散列的
```

## 单分派反函数

In [55]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
                                  
print(htmlize({1,2,3}))
print(htmlize(abs))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>


In [56]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print(htmlize({1,2,3}))
print(htmlize(abs))

print(htmlize("xxxxxxxxxxxxxxxxxxxxxxxxxxx"))

print(htmlize(1111))

print(htmlize((1,2,3,4,5)))
print(htmlize([6,7,8,9]))


<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>xxxxxxxxxxxxxxxxxxxxxxxxxxx</p>
<pre>1111 (0x457)</pre>
<ul>
<li><pre>1 (0x1)</pre></li>
<li><pre>2 (0x2)</pre></li>
<li><pre>3 (0x3)</pre></li>
<li><pre>4 (0x4)</pre></li>
<li><pre>5 (0x5)</pre></li>
</ul>
<ul>
<li><pre>6 (0x6)</pre></li>
<li><pre>7 (0x7)</pre></li>
<li><pre>8 (0x8)</pre></li>
<li><pre>9 (0x9)</pre></li>
</ul>


# 叠放装饰器

# 参数化装饰器

In [59]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    print('running main()')
    print('registry ->', registry)
    
f1()

running register(<function f1 at 0x05AF7D68>)
running f1()
running main()
registry -> [<function f1 at 0x05AF7D68>]


In [61]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s) -> decorate(%s)' % (active, func))
        
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('funning f3()')
    
    
    
            

running register(active=False) -> decorate(<function f1 at 0x07382540>)
running register(active=True) -> decorate(<function f2 at 0x07382ED0>)
